In [62]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
import urllib
import codecs

In [63]:
#soup = BeautifulSoup(r, 'html.parser')

censorship_phrase_utf8 = '根据相关法律法规和政策'
censorship_phrase_decoded = codecs.encode(censorship_phrase_utf8.decode('utf8'), 'unicode_escape')

def is_it_censored(keyword):
    r = urllib.urlopen('http://s.weibo.com/weibo/%s&Refer=index' % keyword).read()
    if censorship_phrase_decoded in r:
        return True
    else:
        return False
    #soup = BeautifulSoup(r, 'html.parser')

In [64]:
is_it_censored("hello")

False

In [65]:
is_it_censored("lxb")

True